# 03｜Step 2（模型）：值得佈陣打者（落點集中性）分類模型 + 模型比較

本 Notebook 建立打者層級分類模型，用行為與情境化特徵預測「是否屬於落點高度集中（值得佈陣）」。

模型：Logistic Regression（可解釋基準） vs Random Forest / XGBoost（非線性驗證）

輸出：
- `step2_model_comparison_metrics.csv`
- `step2_model_comparison_test_predictions.csv`

⚠️ 請先跑 Notebook 01，確保打者層級特徵檔存在（程式內會讀取）。

In [ ]:
#（Colab 可選）掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# =========================
# Step 2（學術版 / 乾淨無洩漏版）
# 目的：用「打者行為特徵」預測打者是否屬於「落點高度集中（適合佈陣）」族群
#
# 三個修正（你要求的）：
# (1) y 更嚴格：用 top3_share 的分位數（預設 top 25%）定義 y=1
# (2) 防止標籤洩漏：X 不包含任何用來定義 y 的落點集中指標（top1/top3/entropy 一律不放）
# (3) 任務定義更像論文：辨識「極端集中」打者（hard classification），不是「大多數都是1」
#
# 你會得到：
# - step2_clean_batter_model_dataset.csv（含 y、閾值、特徵）
# - step2_clean_logit_coefficients.csv（係數表）
# - step2_clean_test_predictions.csv（測試集預測，用於挑 case study）
# =========================

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report, confusion_matrix

# ========= 0) 路徑設定（改這裡） =========
DATA_DIR = "/content/drive/MyDrive/CPBL_csv_tables_UTF8_BOM"  # <- 改成你的資料夾
PA_PATH   = os.path.join(DATA_DIR, "PA.csv")
FEAR_PATH = os.path.join(DATA_DIR, "batter_ctx_fear_pitch_gb_topzones_full.csv")  # 有就用，沒有就跳過

OUT_DATA = os.path.join(DATA_DIR, "step2_clean_batter_model_dataset.csv")
OUT_COEF = os.path.join(DATA_DIR, "step2_clean_logit_coefficients.csv")
OUT_PRED = os.path.join(DATA_DIR, "step2_clean_test_predictions.csv")

# ========= 1) 讀檔 =========
pa = pd.read_csv(PA_PATH, encoding="utf-8-sig", low_memory=False)

need = ["batterName", "trajectory", "locationCode", "bases"]
for c in need:
    if c not in pa.columns:
        raise ValueError(f"PA.csv missing required column: {c}")

pa["batterName"] = pa["batterName"].astype(str).str.strip()
pa["trajectory"] = pa["trajectory"].astype(str).str.strip().str.upper()
pa["locationCode_int"] = pd.to_numeric(pa["locationCode"], errors="coerce")

# ========= 2) 壘況分組（bases_group） =========
def bases_group(b):
    if b is None or (isinstance(b, float) and np.isnan(b)):
        return "UNK"
    s = str(b).strip()
    if s == "" or s.lower() == "nan":
        return "UNK"
    if s in {"0","---","EMPTY","NONE","none","None"}:
        return "Empty"
    u = s.upper()
    # 很保守：只要字串帶有 2 / 3（或 2B/3B）就視作得點圈
    if ("2" in s) or ("3" in s) or ("2B" in u) or ("3B" in u):
        return "RISP"
    return "OnBase"

pa["bases_group"] = pa["bases"].apply(bases_group)

# ========= 3) 取 GB 且有落點 =========
is_gb = pa["trajectory"].fillna("").str.startswith("G")
gb = pa[is_gb & pa["locationCode_int"].notna()].copy()

# ========= 4) 落點方向（只用來做 X 的方向比例，不用來做 y） =========
LEFT_ZONES   = {5, 45, 6, 56, 7}
CENTER_ZONES = {1, 2, 12, 23, 4, 34}
RIGHT_ZONES  = {3, 13, 14, 24, 8, 9}

def zone_dir(z):
    try:
        z = int(z)
    except:
        return "UNK"
    if z in LEFT_ZONES: return "L"
    if z in CENTER_ZONES: return "C"
    if z in RIGHT_ZONES: return "R"
    return "O"

gb["dir"] = gb["locationCode_int"].apply(zone_dir)

# ========= 5) 先做 batter-level 基礎統計 =========
MIN_PA = 100
MIN_GB = 30  # 避免小樣本落點集中度不穩

b_pa = pa.groupby("batterName").size().rename("PA").reset_index()
b_gb = gb.groupby("batterName").size().rename("GB_inplay_n").reset_index()

df = b_pa.merge(b_gb, on="batterName", how="left")
df["GB_inplay_n"] = df["GB_inplay_n"].fillna(0).astype(int)
df["GB_rate"] = np.where(df["PA"]>0, df["GB_inplay_n"]/df["PA"], np.nan)

# 方向比例特徵（X）
dir_pivot = (gb.pivot_table(index="batterName", columns="dir", values="locationCode_int",
                            aggfunc="count", fill_value=0))
for col in ["L","C","R","O","UNK"]:
    if col not in dir_pivot.columns:
        dir_pivot[col] = 0
dir_pivot = dir_pivot[["L","C","R","O","UNK"]].copy()
dir_pivot["dir_total"] = dir_pivot.sum(axis=1)

for col in ["L","C","R"]:
    dir_pivot[f"dir_{col}_share"] = np.where(dir_pivot["dir_total"]>0, dir_pivot[col]/dir_pivot["dir_total"], np.nan)

dir_feat = dir_pivot.reset_index()[["batterName","dir_L_share","dir_C_share","dir_R_share"]]
df = df.merge(dir_feat, on="batterName", how="left")

# 情境穩定性特徵（X）：RISP 下 GB%（不是落點集中度）
gb_risp = gb[gb["bases_group"]=="RISP"].copy()
risp_gb_n = gb_risp.groupby("batterName").size().rename("GB_RISP_n").reset_index()
df = df.merge(risp_gb_n, on="batterName", how="left")
df["GB_RISP_n"] = df["GB_RISP_n"].fillna(0).astype(int)
df["GB_RISP_rate"] = np.where(df["GB_inplay_n"]>0, df["GB_RISP_n"]/df["GB_inplay_n"], np.nan)

# ========= 6) （只用於定義 y）計算 top3_share =========
# 注意：top3_share 只用於 y 的定義，後面不會放進 X
TOPK = 3
def topk_share(series, k=3):
    vc = series.value_counts()
    if vc.sum() == 0:
        return np.nan
    return vc.head(k).sum() / vc.sum()

top3 = (gb.groupby("batterName")["locationCode_int"]
          .apply(lambda s: topk_share(s, TOPK))
          .rename("top3_share_for_label")
          .reset_index())

df = df.merge(top3, on="batterName", how="left")

# ========= 7) 建模母體（先過濾樣本量） =========
model_df = df[(df["PA"]>=MIN_PA) & (df["GB_inplay_n"]>=MIN_GB) & df["top3_share_for_label"].notna()].copy()

# ========= 8) y：用分位數定義「極端集中」打者 =========
# 例：TOP_Q = 0.75 -> top 25% 為 y=1
TOP_Q = 0.75
theta = float(model_df["top3_share_for_label"].quantile(TOP_Q))
model_df["y_shiftable"] = (model_df["top3_share_for_label"] >= theta).astype(int)

print("建模打者數:", len(model_df))
print("使用分位數 TOP_Q =", TOP_Q, "=> theta =", round(theta, 3))
print("y=1 比例:", round(model_df["y_shiftable"].mean(), 3))

# ========= 9) X：只用「行為特徵」，不含落點集中度 =========
feature_cols = [
    "PA", "GB_inplay_n", "GB_rate",
    "dir_L_share", "dir_C_share", "dir_R_share",
    "GB_RISP_rate", "GB_RISP_n",
]

# 加入 fear 特徵（如果存在）—這是行為特徵，不是落點集中度
if os.path.exists(FEAR_PATH):
    fear = pd.read_csv(FEAR_PATH, encoding="utf-8-sig", low_memory=False)
    fear["batterName"] = fear["batterName"].astype(str).str.strip()
    if "fear_whiff_rate" in fear.columns:
        fear_all = fear.groupby("batterName")["fear_whiff_rate"].mean().rename("fear_whiff_mean").reset_index()
        fear_risp = (fear[fear["bases_group"]=="RISP"]
                     .groupby("batterName")["fear_whiff_rate"]
                     .mean()
                     .rename("fear_whiff_RISP_mean")
                     .reset_index())
        model_df = model_df.merge(fear_all, on="batterName", how="left").merge(fear_risp, on="batterName", how="left")
        feature_cols += ["fear_whiff_mean", "fear_whiff_RISP_mean"]
    else:
        print("FEAR 表沒有 fear_whiff_rate，略過 fear 特徵")
else:
    print("找不到 FEAR_PATH，略過 fear 特徵")

X = model_df[feature_cols].copy()
y = model_df["y_shiftable"].copy()

# ========= 10) 以「打者」為單位切 train/test =========
train_idx, test_idx = train_test_split(
    model_df.index,
    test_size=0.25,
    random_state=42,
    stratify=y
)

X_train, X_test = X.loc[train_idx], X.loc[test_idx]
y_train, y_test = y.loc[train_idx], y.loc[test_idx]

# ========= 11) Logistic Regression pipeline =========
pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced", solver="lbfgs"))
])

pipe.fit(X_train, y_train)

proba = pipe.predict_proba(X_test)[:, 1]
pred  = (proba >= 0.5).astype(int)

auc = roc_auc_score(y_test, proba)
f1  = f1_score(y_test, pred)
acc = accuracy_score(y_test, pred)

print("\n=== Step 2（乾淨無洩漏版）Logistic Regression ===")
print("ROC-AUC:", round(auc, 3))
print("Accuracy:", round(acc, 3))
print("F1:", round(f1, 3))
print("\nConfusion matrix:\n", confusion_matrix(y_test, pred))
print("\nReport:\n", classification_report(y_test, pred, digits=3))

# ========= 12) 係數表（可寫論文：標準化後係數可比） =========
coef = pipe.named_steps["clf"].coef_.ravel()
coef_df = pd.DataFrame({"Feature": feature_cols, "Coefficient": coef}).sort_values("Coefficient", ascending=False)
coef_df.to_csv(OUT_COEF, index=False, encoding="utf-8-sig")
print("Saved coefficients:", OUT_COEF)
display(coef_df)

# ========= 13) 輸出建模資料（含 theta 與 top3_share_for_label，但注意：這些不進 X） =========
save_df = model_df.copy()
save_df["label_theta"] = theta
save_df.to_csv(OUT_DATA, index=False, encoding="utf-8-sig")
print("Saved dataset:", OUT_DATA)

# ========= 14) 輸出測試集預測（挑 case study 超好用） =========
result = save_df.loc[test_idx, ["batterName","PA","GB_inplay_n","GB_rate","top3_share_for_label","y_shiftable"]].copy()
result["pred_proba"] = proba
result["pred_label"] = pred
result = result.sort_values("pred_proba", ascending=False)

result.to_csv(OUT_PRED, index=False, encoding="utf-8-sig")
print("Saved test predictions:", OUT_PRED)
display(result.head(25))

# ========= 15) （可選）快速 sanity check：看看 y=1 的 top3_share 分布是否真的較高 =========
print("\nLabel sanity check:")
print("top3_share median | y=0:", round(save_df.loc[save_df.y_shiftable==0, "top3_share_for_label"].median(), 3))
print("top3_share median | y=1:", round(save_df.loc[save_df.y_shiftable==1, "top3_share_for_label"].median(), 3))

## 3.2 模型比較（Logistic vs RF vs XGBoost）

In [ ]:
# ============================================================
# Step 2 三模型比較（Logit + RandomForest + XGBoost）
# - 公平比較：同一份資料、同一個 y、同一個 train/test split
# - 指標：ROC-AUC、PR-AUC、F1、Accuracy
# - 輸出：comparison_metrics.csv、test_predictions.csv、
#         logit_coefficients.csv、rf_importance.csv、xgb_importance.csv
#
# 你只要改：DATA_DIR 位置
# ============================================================

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    roc_auc_score, average_precision_score, f1_score, accuracy_score,
    confusion_matrix, classification_report
)

# ---- XGBoost（Colab 通常可用；沒有就安裝）----
try:
    from xgboost import XGBClassifier
except Exception:
    !pip -q install xgboost
    from xgboost import XGBClassifier

# ======================
# 0) 路徑設定（改這裡）
# ======================
DATA_DIR = "/content/drive/MyDrive/CPBL_csv_tables_UTF8_BOM"  # <- 改成你的資料夾
DATA_PATH = os.path.join(DATA_DIR, "step2_clean_batter_model_dataset.csv")  # Step2 乾淨版資料
OUT_METRICS = os.path.join(DATA_DIR, "step2_model_comparison_metrics.csv")
OUT_PRED    = os.path.join(DATA_DIR, "step2_model_comparison_test_predictions.csv")
OUT_LOGIT_COEF = os.path.join(DATA_DIR, "step2_logit_coefficients.csv")
OUT_RF_IMP     = os.path.join(DATA_DIR, "step2_rf_feature_importance.csv")
OUT_XGB_IMP    = os.path.join(DATA_DIR, "step2_xgb_feature_importance.csv")

# ======================
# 1) 讀取資料
# ======================
df = pd.read_csv(DATA_PATH, encoding="utf-8-sig", low_memory=False)

if "y_shiftable" not in df.columns:
    raise ValueError("Dataset missing y_shiftable. 請確認你用的是 step2_clean_batter_model_dataset.csv")

# 你當初 Step2 用到的 feature_cols（若你有加新特徵，請在下面補進去）
# 注意：不要把 top3_share_for_label / label_theta / pred_proba 類的放進來
DEFAULT_FEATURE_COLS = [
    "PA", "GB_inplay_n", "GB_rate",
    "dir_L_share", "dir_C_share", "dir_R_share",
    "GB_RISP_rate", "GB_RISP_n",
    "fear_whiff_mean", "fear_whiff_RISP_mean"
]
feature_cols = [c for c in DEFAULT_FEATURE_COLS if c in df.columns]

if len(feature_cols) < 4:
    raise ValueError(f"可用特徵太少：{feature_cols}。請確認資料欄位，或把你的特徵欄位名補進 DEFAULT_FEATURE_COLS")

X = df[feature_cols].copy()
y = df["y_shiftable"].astype(int).copy()

print("建模打者數:", len(df))
print("y=1 比例:", round(y.mean(), 3))
print("使用特徵:", feature_cols)

# ======================
# 2) 固定切分（公平比較）
# ======================
train_idx, test_idx = train_test_split(
    df.index, test_size=0.25, random_state=42, stratify=y
)

X_train, X_test = X.loc[train_idx], X.loc[test_idx]
y_train, y_test = y.loc[train_idx], y.loc[test_idx]

print("Train size:", len(train_idx), "Test size:", len(test_idx))
print("Train y=1:", round(y_train.mean(), 3), "Test y=1:", round(y_test.mean(), 3))

# ======================
# 3) 建立三個模型（同樣 imputer；Logit 需要 scaler）
# ======================
logit = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=5000, class_weight="balanced", solver="lbfgs"))
])

rf = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("clf", RandomForestClassifier(
        n_estimators=800,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced_subsample",
        min_samples_leaf=2
    ))
])

# 針對小樣本：XGB 不要太深，避免 overfit
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
scale_pos_weight = (neg / max(pos, 1))

xgb = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("clf", XGBClassifier(
        n_estimators=600,
        max_depth=3,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        random_state=42,
        eval_metric="logloss",
        scale_pos_weight=scale_pos_weight
    ))
])

models = {
    "LogisticRegression": logit,
    "RandomForest": rf,
    "XGBoost": xgb
}

# ======================
# 4) 評估函數
# ======================
def eval_model(name, model):
    model.fit(X_train, y_train)
    proba = model.predict_proba(X_test)[:, 1]
    pred = (proba >= 0.5).astype(int)

    auc = roc_auc_score(y_test, proba)
    pr  = average_precision_score(y_test, proba)  # PR-AUC
    f1  = f1_score(y_test, pred)
    acc = accuracy_score(y_test, pred)

    print("\n====", name, "====")
    print("ROC-AUC:", round(auc, 3))
    print("PR-AUC :", round(pr, 3))
    print("Acc   :", round(acc, 3))
    print("F1    :", round(f1, 3))
    print("Confusion:\n", confusion_matrix(y_test, pred))
    print(classification_report(y_test, pred, digits=3))

    return auc, pr, acc, f1, proba, pred

rows = []
pred_pack = []

for name, model in models.items():
    auc, pr, acc, f1, proba, pred = eval_model(name, model)
    rows.append({"Model": name, "ROC_AUC": auc, "PR_AUC": pr, "Accuracy": acc, "F1": f1})
    pred_pack.append((name, proba, pred))

metrics_df = pd.DataFrame(rows).sort_values("ROC_AUC", ascending=False)
metrics_df.to_csv(OUT_METRICS, index=False, encoding="utf-8-sig")
print("\nSaved metrics ->", OUT_METRICS)
display(metrics_df)

# ======================
# 5) 輸出 test 預測（挑 case study 用）
# ======================
base_cols = ["batterName", "PA", "GB_inplay_n", "GB_rate", "top3_share_for_label", "y_shiftable"]
base_cols = [c for c in base_cols if c in df.columns]

pred_df = df.loc[test_idx, base_cols].copy() if base_cols else df.loc[test_idx, ["y_shiftable"]].copy()
for name, proba, pred in pred_pack:
    pred_df[f"{name}_proba"] = proba
    pred_df[f"{name}_pred"] = pred

pred_df = pred_df.sort_values([c for c in pred_df.columns if c.endswith("_proba")][0], ascending=False)
pred_df.to_csv(OUT_PRED, index=False, encoding="utf-8-sig")
print("Saved test predictions ->", OUT_PRED)
display(pred_df.head(25))

# ======================
# 6) 特徵解釋輸出（Logit 係數 / RF & XGB 重要性）
# ======================
# ---- Logit 係數 ----
logit_fit = models["LogisticRegression"].fit(X_train, y_train)
coef = logit_fit.named_steps["clf"].coef_.ravel()
coef_df = pd.DataFrame({"Feature": feature_cols, "Coefficient": coef}).sort_values("Coefficient", ascending=False)
coef_df.to_csv(OUT_LOGIT_COEF, index=False, encoding="utf-8-sig")
print("Saved logit coefficients ->", OUT_LOGIT_COEF)
display(coef_df)

# ---- RF 重要性 ----
rf_fit = models["RandomForest"].fit(X_train, y_train)
rf_imp = rf_fit.named_steps["clf"].feature_importances_
rf_imp_df = pd.DataFrame({"Feature": feature_cols, "Importance": rf_imp}).sort_values("Importance", ascending=False)
rf_imp_df.to_csv(OUT_RF_IMP, index=False, encoding="utf-8-sig")
print("Saved RF importances ->", OUT_RF_IMP)
display(rf_imp_df.head(15))

# ---- XGB 重要性 ----
xgb_fit = models["XGBoost"].fit(X_train, y_train)
xgb_imp = xgb_fit.named_steps["clf"].feature_importances_
xgb_imp_df = pd.DataFrame({"Feature": feature_cols, "Importance": xgb_imp}).sort_values("Importance", ascending=False)
xgb_imp_df.to_csv(OUT_XGB_IMP, index=False, encoding="utf-8-sig")
print("Saved XGB importances ->", OUT_XGB_IMP)
display(xgb_imp_df.head(15))

# ======================
# 7) （可選）交叉驗證 AUC：讓論文更像研究
# ======================
print("\n=== 5-fold Stratified CV（ROC-AUC）===")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for name, model in models.items():
    # 用 whole dataset 做 CV（仍然不含 label leakage 特徵）
    aucs = cross_val_score(model, X, y, cv=cv, scoring="roc_auc")
    print(f"{name:>18}  mean AUC={aucs.mean():.3f}  std={aucs.std():.3f}")